## Pré-Processamento de Dados

In [56]:
import pandas as pd
import json
#import datetime

In [57]:
df = pd.read_json("./datas/tweet.json", lines = True)
df.head(10)

,created_at,likes_count,screen_name,tweet
0,2021-12-16 23:23:06,186,bbcbrasil,"Para especialistas, suicídios de donas de casa..."
1,2021-12-14 15:30:17,48183,CrimesReais,"Assédio, violência, envenenamento, drogas ilíc..."
2,2021-12-17 14:22:38,7,mdhbrasil,"Promover a autonomia financeira das mulheres, ..."
3,2021-12-17 18:55:07,3,policiacivilrs,"Em Bagé, homem de 43 anos é preso, preventivam..."
4,2021-12-17 18:50:19,0,nochldlefbehind,"@aychawane eu achei uma porcaria, real. projet..."
5,2021-12-17 18:50:07,0,4Cabocla,@frulanis Na violencia doméstica onde vi dezen...
6,2021-12-17 18:48:24,0,descriartepod,Neste episódio aproveitamos a obra Sol da Manh...
7,2021-12-17 18:46:02,0,extensaoUFRJ,Os vencedores foram divulgados nesta terça-fei...
8,2021-12-17 18:41:53,2,ziripitico,E é daí que vem a principal diferença da liga ...
9,2021-12-17 18:38:18,1,T0neZe,@AlexiLapas Podíamos deixar de vez de falar em...


In [58]:
#pip install nltk

In [59]:
# NLKT - Natural Language Toolkit
import nltk
#nltk.download()

In [60]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer

In [61]:
# Funções de pré-processamento (limpeza de dados)
tweet_tokenizer = TweetTokenizer()

def DataCleansing(instance):
  # Remove caracteres, artificios indesejados (links, virgula, pontuação, etc...)
  instance = re.sub(r"http\S+", "", instance).lower().replace('.', '').replace(';', '').replace('-', '').replace(':', '').replace(')', '')
  stopwords = set(nltk.corpus.stopwords.words('portuguese'))
  words = [i for i in instance.split() if not i in stopwords]
  return (" ".join(words))

def PreprocessTweet(tweet):
  tweet = BeautifulSoup(tweet, "html.parser").get_text()
  tweet = re.sub(r"[^a-zA-Zà-úÀ-Ú0-9]+", " ", tweet.lower())
  return tweet

In [62]:
# Realizando a limpeza dos dados
tweets = [DataCleansing(i) for i in df.tweet]

In [63]:
df['clean_data'] = tweets

In [64]:
df['preprocessed_data'] = [PreprocessTweet(tweet) for tweet in df.clean_data]

## Processamento de Dataframe
### - Contagem de quantidade de Palavras

  Identificar as palavras que mais são usadas ou são associadas quando o tema é violência doméstica assédio e feminicídio.

In [65]:
# Importação de biblioteca para contar o número de vezes que uma palavra ocorre
from sklearn.feature_extraction.text import CountVectorizer


In [66]:
cv = CountVectorizer()
counts = cv.fit_transform(df.preprocessed_data)

In [67]:
# Captura as palavras de todos os tweets
cv.get_feature_names()

['001',
 '0genos',
 '12',
 '124',
 '14',
 '144',
 '148',
 '19',
 '20',
 '202',
 '2020',
 '2021',
 '24',
 '25',
 '26',
 '290',
 '43',
 '58',
 '800',
 '90daysfiance',
 '90diasparacasar',
 'abalada',
 'abusos',
 'acaba',
 'acabam',
 'acertando',
 'achei',
 'acompanha',
 'acompanhar',
 'acontecendo',
 'acusado',
 'acusados',
 'acusação',
 'adam',
 'admitido',
 'adorava',
 'afastados',
 'afastamento',
 'afazenda',
 'afirma',
 'agir',
 'agora',
 'agredido',
 'agressor',
 'agressores',
 'agressão',
 'agressões',
 'aguentar',
 'ainda',
 'ajuda',
 'ajudar',
 'alegações',
 'alexilapas',
 'alfa',
 'alguém',
 'alvoroço',
 'além',
 'amaral',
 'amber',
 'ameaça',
 'ameaças',
 'amor',
 'amoroso',
 'and',
 'ano',
 'anos',
 'antigo',
 'aoyama',
 'apaixonando',
 'apareceu',
 'aplauda',
 'aplicadas',
 'apoiamos',
 'apoio',
 'aposto',
 'aprendir',
 'aproveita',
 'aproveitamos',
 'aproximar',
 'aqula',
 'aqulas',
 'arma',
 'arrecadando',
 'artigo',
 'artigos',
 'artistas',
 'assassinato',
 'assassino',
 'a

In [68]:
counts.toarray()

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [69]:
word_count = pd.DataFrame(cv.get_feature_names(), columns = ['word'])
word_count['count'] = counts.sum(axis=0).tolist()[0]
word_count = word_count.sort_values('count', ascending = False).reset_index(drop = True)

In [70]:
words = [index for index, row in word_count.iterrows() if row['word'] == 'violência' or row['word'] == 'doméstica' or row['word'] == 'assédio' or row['word'] == 'feminicídio']
df_wordcount = word_count.drop(words)
df_wordcount.reset_index(inplace=True, drop=True)
df_wordcount

,word,count
0,mulheres,17
1,contra,13
2,pra,11
3,vítima,11
4,anos,11
5,day,10
6,caso,7
7,mulher,7
8,quase,7
9,cara,7
